<a href="https://colab.research.google.com/github/heber-augusto/sus-kpis-analysis/blob/main/sia/indicadores_monitor_rosa_sia_pa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apresentação
Este notebook foi desenolvido como parte do projeto Monitor Rosa, um conjunto de ferramentas, softwares e outros artefatos cujo principal objetivo é melhorar o cenário de diagnósticos de câncer de mama no Brasil, idealmente melhorando diminuindo a proporção de diagnósticos tardios/diagnósticos precoces.

Os arquivos utilizados neste notebook foram coletados através dos scripts presentes [neste repositório](https://github.com/heber-augusto/devops-pysus-get-files). Os arquios foram armazenados em um bucket do Google Storage no formato parquet compactados (gzip).

# Instalação de bibliotecas e pacotes para leitura de arquivos

## Configurações iniciais para conectar com bucket no Google Storage
 - Autenticação do Google Colab
 - Definição do nome do projeto

 Para a execução dos comandos desta seção, o arquio gcp-leitura.json deve ser inserido na raiz do colab.

In [1]:
from google.colab import auth
auth.authenticate_user()

# id do projeto
project_id = 'teak-ellipse-317015'
# id do bucket dentro do projeto
bucket_id = 'observatorio-oncologia'

# nome da pasta local para mapear
local_folder_name = 'colab'

# nome da pasta do projeto
project_folder_name = 'monitor'

!gcloud config set project {project_id}

Updated property [core/project].


To take a quick anonymous survey, run:
  $ gcloud survey



## Instalação para garantir montagem da pasta no bucket
Instalação do gcsfuse para mapear pasta do bucket no google colab

In [2]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-bionic main" > /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add -
!apt -qq update
!apt -qq install gcsfuse

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  61878      0 --:--:-- --:--:-- --:--:-- 61878
OK
71 packages can be upgraded. Run 'apt list --upgradable' to see them.
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  gcsfuse
0 upgraded, 1 newly installed, 0 to remove and 71 not upgraded.
Need to get 11.6 MB of archives.
After this operation, 27.4 MB of additional disk space will be used.
Selecting previously unselected package gcsfuse.
(Reading database ... 155653 files and directories currently installed.)
Preparing to unpack .../gcsfuse_0.41.4_amd64.deb ...
Unpacking gcsfuse (0.41.4) ...
Setting up gcsfuse (0.41.4) ...


## Nome/caminho do arquivo json contendo credenciais de acesso ao storage do Google

In [4]:
serice_account_json = '/content/gcp-leitura.json'

## Montagem do bucket em uma pasta local do google colab

In [5]:
!mkdir {local_folder_name}
!gcsfuse --key-file {serice_account_json} --implicit-dirs {bucket_id} {local_folder_name}

mkdir: cannot create directory ‘colab’: File exists
2022/07/28 10:55:44.710291 Start gcsfuse/0.41.4 (Go version go1.17.6) for app "" using mount point: /content/colab
2022/07/28 10:55:44.726460 Opening GCS connection...
2022/07/28 10:55:45.127626 Mounting file system "observatorio-oncologia"...
2022/07/28 10:55:45.165279 File system has been successfully mounted.


## Função para facilitar coleta de arquivo

In [6]:
import glob
import os

def get_files(state, year, month, file_type, file_group):
    initial_path = os.path.join(r'/content/',local_folder_name,project_folder_name)
    internal_folder = f"""{state}/{year}/{month}/{file_type}/{file_group}"""
    # print(f"{initial_path}/{internal_folder}/*.parquet.gzip")
    return glob.glob(f"{initial_path}/{internal_folder}/*.parquet.gzip")    

# SIA PA: Análise exploratória

## Informações sobre filtros pertinentes ao contexto de câncer de mama:

### SIH: realizar filtro através da variável DIAG_PRINC (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### RHC: realizar filtro através da variável LOCTUDET (3 caracteres)
 * Filtro: C50

### SIA – APAC de Quimioterapia e Radioterapia (AQ e AR)
Realizar filtro através da variável AP_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

### SIA – Procedimentos ambulatoriais (PA)

Os arquivos de procedimentos ambulatoriais são um pouco diferentes por um motivo: a pessoa já pode ter o diagnóstico e está realizando um procedimento OU a pessoa está realizando um exame com finalidade diagnóstica (mamografia, ultrassonografia, etc). Então, neste caso, podemos pensar em dois filtros:

Realizar filtro através da variável PA_CIDPRI (4 caracteres)
 * Filtro: C500, C501, C502, C503, C504, C505, C506, C508 e C509

Realizar filtro através da variável do código de procedimento ambulatorial “PA_PROC_ID” (10 caracteres)
 * Filtros:
  * 201010569	BIOPSIA/EXERESE DE NÓDULO DE MAMA
  * 201010585	PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA
  * 201010607	PUNÇÃO DE MAMA POR AGULHA GROSSA
  * 203010035	EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)
  * 203010043	EXAME CITOPATOLOGICO DE MAMA
  * 203020065	EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA
  * 203020073	EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA
  * 205020097	ULTRASSONOGRAFIA MAMARIA BILATERAL
  * 208090037	CINTILOGRAFIA DE MAMA (BILATERAL)
  * 204030030	MAMOGRAFIA
  * 204030188	MAMOGRAFIA BILATERAL PARA RASTREAMENTO

## Bibliotecas utilizadas na análise exploratória

In [8]:
import pandas as pd
import numpy as np 

## Variáveis de filtro

In [19]:
# filtro pelo cid
cid_filter = ['C500', 'C501', 'C502', 'C503', 'C504', 'C505', 'C506', 'C508', 'C509']

# dicionario de procedimentos
proc_id_dict = {
    '0201010569': 'BIOPSIA/EXERESE DE NÓDULO DE MAMA',
    '0201010585': 'PUNÇÃO ASPIRATIVA DE MAMA POR AGULHA FINA',
    '0201010607': 'PUNÇÃO DE MAMA POR AGULHA GROSSA',
    '0203010035': 'EXAME DE CITOLOGIA (EXCETO CERVICO-VAGINAL E DE MAMA)',
    '0203010043': 'EXAME CITOPATOLOGICO DE MAMA',
    '0203020065': 'EXAME ANATOMOPATOLOGICO DE MAMA - BIOPSIA',
    '0203020073': 'EXAME ANATOMOPATOLOGICO DE MAMA - PECA CIRURGICA',
    '0205020097': 'ULTRASSONOGRAFIA MAMARIA BILATERAL',
    '0208090037': 'CINTILOGRAFIA DE MAMA (BILATERAL)',
    '0204030030': 'MAMOGRAFIA',
    '0204030188': 'MAMOGRAFIA BILATERAL PARA RASTREAMENTO'
    }
proc_id_filter = list(proc_id_dict.keys())


## Função de filtro para arquivo SIA PA

In [21]:
def filter_cancer_content(df):
    """

    """
    return df[df.PA_CIDPRI.isin(cid_filter) & \
              df.PA_PROC_ID.isin(proc_id_filter)]

## SIA PA: Teste de leitura de arquivo e erificação de colunas

In [9]:
file_paths = get_files('SP','2014','05','SIA','PA')
file_paths

['/content/colab/monitor/SP/2014/05/SIA/PA/PASP1405a.parquet.gzip',
 '/content/colab/monitor/SP/2014/05/SIA/PA/PASP1405b.parquet.gzip']

In [11]:
df = pd.read_parquet(file_paths[1])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3924906 entries, 0 to 3924905
Data columns (total 58 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   PA_CODUNI   object
 1   PA_GESTAO   object
 2   PA_CONDIC   object
 3   PA_UFMUN    object
 4   PA_REGCT    object
 5   PA_INCOUT   object
 6   PA_INCURG   object
 7   PA_TPUPS    object
 8   PA_TIPPRE   object
 9   PA_MN_IND   object
 10  PA_CNPJCPF  object
 11  PA_CNPJMNT  object
 12  PA_CNPJ_CC  object
 13  PA_MVM      object
 14  PA_CMP      object
 15  PA_PROC_ID  object
 16  PA_TPFIN    object
 17  PA_SUBFIN   object
 18  PA_NIVCPL   object
 19  PA_DOCORIG  object
 20  PA_AUTORIZ  object
 21  PA_CNSMED   object
 22  PA_CBOCOD   object
 23  PA_MOTSAI   object
 24  PA_OBITO    object
 25  PA_ENCERR   object
 26  PA_PERMAN   object
 27  PA_ALTA     object
 28  PA_TRANSF   object
 29  PA_CIDPRI   object
 30  PA_CIDSEC   object
 31  PA_CIDCAS   object
 32  PA_CATEND   object
 33  PA_IDADE    object
 34  IDADEMIN    object

In [22]:
filter_cancer_content(df)

,PA_CODUNI,PA_GESTAO,PA_CONDIC,PA_UFMUN,PA_REGCT,PA_INCOUT,PA_INCURG,PA_TPUPS,PA_TIPPRE,PA_MN_IND,...,NU_PA_TOT,PA_INDICA,PA_CODOCO,PA_FLQT,PA_FLER,PA_ETNIA,PA_VL_CF,PA_VL_CL,PA_VL_INC,PA_SRV_C
308898,6697313,351880,PG,351880,0000,0000,0000,39,20,M,...,24.0,5,1,K,0,,0.0,0.0,0.0,
309089,6697313,351880,PG,351880,0000,0000,0000,39,20,M,...,24.0,5,1,K,0,,0.0,0.0,0.0,
413124,2038463,350635,PG,350635,0000,0000,0000,02,50,M,...,24.2,5,1,K,0,,0.0,0.0,0.0,
413130,2038463,350635,PG,350635,0000,0000,0000,02,50,M,...,24.2,5,1,K,0,,0.0,0.0,0.0,
413133,2038463,350635,PG,350635,0000,0000,0000,02,50,M,...,24.2,5,1,K,0,,0.0,0.0,0.0,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3922302,2083116,352900,PG,352900,0000,0000,0000,05,61,I,...,68.43,5,1,K,0,,0.0,0.0,0.0,
3922508,2035081,354870,PG,354870,0000,0000,0000,62,50,M,...,68.43,5,1,S,0,,0.0,0.0,0.0,
3922529,2027356,354870,PG,354870,0000,0000,0000,05,50,M,...,68.43,5,1,R,0,,0.0,0.0,0.0,
3924127,2077590,355030,MN,355030,7101,0000,0000,07,61,I,...,24.0,5,1,R,0,,0.0,0.0,0.0,


In [18]:
df.PA_PROC_ID

0          0202010694
1          0202010210
2          0202010694
3          0305010107
4          0202010694
              ...    
3924901    0205020097
3924902    0205020143
3924903    0211060259
3924904    0302050019
3924905    0205020143
Name: PA_PROC_ID, Length: 3924906, dtype: object